In [197]:
import random

from tqdm import tqdm
from datasets import load_from_disk, DatasetDict

In [47]:
wiki_datasets = load_from_disk("/home/ubuntu/workspace/data/wiki_preprocessed_droped")
train_dataset = load_from_disk("/home/ubuntu/workspace/data/train_preprocessed")
wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")

In [54]:
dicts = []
for wiki in tqdm(wiki_datasets):
  wiki_dick = {}
  wiki_dick['text'] = wiki['text']
  wiki_dick['meta'] = {
    'title': wiki['title'],
    'document_id': wiki['document_id']
  }
  dicts.append(wiki_dick)

100%|██████████| 55962/55962 [00:08<00:00, 6317.01it/s]


In [57]:
dict_id = []
for wiki in tqdm(dicts):
  dict_id.append(wiki['meta']['document_id'])

100%|██████████| 55962/55962 [00:00<00:00, 1521587.70it/s]


In [189]:
def drop_not_doc(datasets):
  not_include = []
  for data in tqdm(datasets):
    if data['document_id'] not in dict_id:
      not_include.append(data['document_id'])
  def get_distance(a, b):
    return abs(a - b)
  def change_doc_id(example):
    if example['document_id'] not in not_include:
      return example
    doc = wiki_datasets.get_nearest_examples("text", example['context'][:1024], k=1)
    document_id = doc[1]['document_id'][0]
    title = doc[1]['title'][0]
    context_list = doc[1]['text'][0].split(example['answers']['text'][0])
    distance = len(example['context'])
    location = 0
    for context in context_list:
      location += len(context)
      new_distance = get_distance(location, example['answers']['answer_start'][0])
      if distance > new_distance:
        distance = new_distance
      else:
        location -= len(context)
        break
    example['answers']['answer_start'][0] = location
    example['document_id'] = document_id
    example['title'] = title
    example['context'] = example['answers']['text'][0].join(context_list)
    return example
  return datasets.map(change_doc_id)

In [190]:
new_train_dataset = drop_not_doc(train_dataset['train'])
new_valid_dataset = drop_not_doc(train_dataset['validation'])

100%|██████████| 3952/3952 [00:02<00:00, 1453.08it/s]


  0%|          | 0/3952 [00:00<?, ?ex/s]

/home/ubuntu/workspace/mrc-level2-nlp-01/ipynb_research/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [192]:
data = {
  'train': new_train_dataset,
  'validation': new_valid_dataset
}
train = DatasetDict(data)
train.save_to_disk('train_dataset')

In [200]:
def generate_dpr_dataset(target_dataset, dataset_name):
  dpr_train_datas = []
  def change_score(x):
    x['score'] = 0
    return x
  for data in tqdm(target_dataset):
    train_dict = {}
    train_dict['dataset'] = dataset_name
    train_dict['question'] = data['question']
    train_dict['answers'] = data['answers']['text']
    train_dict['positive_ctxs'] = [{
      'title': data['title'],
      'text': data['context'],
      'score': 1000,
      'title_score': 1,
      'passage_id': data['document_id']
    }]
    negatives = []
    query = data['question']
    scores, retrieved_examples = wiki_datasets.get_nearest_examples("text", query, k=100)
    for index in range(100):
      if retrieved_examples['document_id'][index] == data['document_id']:
        continue
      negative_dict = {
        'title': retrieved_examples['title'][index],
        'text': retrieved_examples['text'][index],
        'score': scores[index],
        'title_score': 0,
        'passage_id': retrieved_examples['document_id'][index]
      }
      negatives.append(negative_dict)
    train_dict['hard_negative_ctxs'] = negatives[:5]
    train_dict['negative_ctxs'] = list(map(change_score, random.sample(negatives[50:], 10)))
    dpr_train_datas.append(train_dict)
  return dpr_train_datas

In [201]:
dpr_train_datas = generate_dpr_dataset(new_train_dataset, 'original_train')
dpr_valid_datas = generate_dpr_dataset(new_valid_dataset, 'original_valid')

  0%|          | 0/3952 [00:00<?, ?it/s]/home/ubuntu/workspace/mrc-level2-nlp-01/ipynb_research/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
100%|██████████| 240/240 [00:16<00:00, 14.14it/s]


In [202]:
dpr_valid_datas[0]

{'dataset': 'original_valid',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'answers': ['한보철강'],
 'positive_ctxs': [{'title': '전효숙',
   'text': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다. 경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. 2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
   'score': 1000,
   'title_score': 1,
   'passage_id': 9027}],
 'hard_negative_ctxs': [{'title': '경남은행 (1970~2014년 기업)',
   'text': '1968년 회사 설립을 발기하고 1970년 창립총회를 거쳐 업무를 시작하였으며 1972년 증권거래소에 주식을 상장하였다. 1997년 IMF 구제금융사건으로 인한 후유증으로 국제

In [ ]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import FAISSDocumentStore

In [ ]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.write_documents(dicts)

In [ ]:
query_model = "klue/bert-base"
passage_model = "klue/bert-base"

In [ ]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=512
)